#                            Stock Analysis and Price Prediction

# Contents
    1. Introduction
    2. Importing libraries
    3. Reading datasets
    4. Building Models
    5. Conclusion

# Introduction
In general, Stock analysis is an assessment of a specific trading instrument, an investment segment, or the entire economy. It is a way of making purchasing and selling decisions for investors and traders. Investors and traders seek to gain an advantage in the markets by making educated choices by analyzing and reviewing past and present results.

This project demonstrates the technique of machine learning called Long Short-Term Memory (LSTM) to forecast market prices and create portfoil optimization using Efficient Frontier. 

For this we will first evaluate the 20 different stocks and pick the top five higher return stocks that will be used to create a optimize portfoilo. Among them, one stock with a higher return and lower risk will be chosen to predict the future stock price.

We will be answering the following question:

1. What was the change in price of the stock over time?
2. What was the monthly return of the stock on average?
3. What was the moving average of the various stocks?
4. What was the correlation between different stocks'?
5. How much value do we put at risk by investing in a particular stock?
6. Portfoilo optimization using Efficient Frontier?
7. How can we attempt to predict future stock behavior using LSTM?

# Importing Libraries

Here are list of all the libraries that are need for this project.

In [ ]:
# here we are importing important libraries
import math
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as sco
import tensorflow as tf
import datetime
from termcolor import colored

from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from pandas_datareader import data as web
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

plt.style.use("Solarize_Light2")
sns.set_style('whitegrid')
%matplotlib inline

# Reading Dataset

In order to carry out inventory analysis, we need historical asset data. There are several information providers, some of which are free, most of which are charged. For this project, we will be  using data from Yahoo's finance website since 2013. We can do this in Python with the pandas-datareader module.

Companies names and their aliases (Ticker) that are used through out our Analysis
    
    1. Apple Inc. = AAPL
    2. Alphabet Inc. = GOOG
    3. Microsoft Corporation = MSFT
    4. Amazon = AMZN
    5. Facebook Inc. = FB
    6. Alibaba Group = BABA
    7. Johnson & Johnson = JNJ
    8. JPMorgon Chase & Co. = JPM
    9. ExxonMobil = XOM
    10.Bank of America = BAC
    11.WalMart Store Inc. = WMT
    12.Wells Fargo & Co. = WFC
    13.Visa Inc. = V
    14.Procter & Gamble Co. = PG
    15.Verizon Communication = VZ
    16.AT&T Inc. = T
    17.UnitedHealth Group Inc. = UNH
    18.Home Depot = HD
    19.Intel = INTC
    20.Oracle = ORCL



In [ ]:
#List of ticker of companies
ticker_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'FB', 'BABA','JNJ', 'JPM', 'XOM', 'BAC', 'WMT', 'WFC', 'V', 'PG', 'VZ', 'T', 'UNH', 'HD', 'INTC', 'ORCL']

# Mapping of Ticker with companies names
CompanyNameMap={
    "AAPL":"Apple Inc",
    "GOOG":"Alphabet Inc",
    "MSFT":"Microsoft Corporation",
    "AMZN":"Amazon",
    "FB":"Facebook Inc",
    "BABA": "Alibaba Group",
    "JNJ":"Johnson & Johnson",
    "JPM":"JPMorgon Chase & Co",
    "XOM": "ExxonMobil",
    "BAC": "Bank of America",
    "WMT": "WalMart Store Inc",
    "WFC":"Wells Fargo & Co",
    "V": "Visa Inc",
    "PG": "Procter & Gamble Co",
    "VZ": "Verizon Communication",
    "T":"AT&T Inc",
    "UNH": "UnitedHealth Group Inc",
    "HD": "Home Depot",
    "INTC": "Intel",
    "ORCL":"Oracle"
}

In [ ]:
# Downloading stocks data since 2013
data = pdr.get_data_yahoo(ticker_list, start = '2015-01-01')

# 1. What was the change in price of the stock over time?

We will start by looking at historical data and then conducting the return calculations to see how well each stock has done.

In [ ]:
# Looking at the portion of historial data
data.tail()

In [ ]:
# Looking at the head of Monthly Adjused Close of companies
monthly_adjusted_close = data['Adj Close'].resample('M').ffill()
monthly_adjusted_close.head()

In [ ]:
#Here is quick summary of each company Monthly Adjusted closing
monthly_adjusted_close.describe()

In [ ]:
# Looking at the head of Monthly volume of companies
monthly_volume = data['Volume'].resample('M').ffill()
monthly_volume.head()

# 2. What was the monthly return of the stock on average?

As we have already gone through each company's historical data, the next thing to do is estimate the returns. We will calculate the returns on monthly rates.

In [ ]:
# Calculating monthly return
monthly_returns = data['Adj Close'].resample('M').ffill().pct_change()
monthly_returns.tail()

In [ ]:
# here we are visualising of Monthly Adjusted Price, Monthly Volume and Monthly Return

fig, axes = plt.subplots(nrows = 20, ncols = 3)
#fig.suptitle('Monthly Adjusted Closing VS Monthly Volume VS Monthly Return')
fig.set_figheight(80)
fig.set_figwidth(15)
plt.subplots_adjust(bottom=0.1, right=1, top=0.9)

columns = list(monthly_adjusted_close) 
  
for i, cols in enumerate(columns,0):
    monthly_adjusted_close[cols].plot(ax = axes[i,0])
    title = CompanyNameMap[ticker_list[i]]
    axes[i,0].set(xlabel='Date', ylabel='Adj Close')
    axes[i,0].set_title(f"{title}")
    axes[i,0].set_ylim(0, 3500)
    monthly_volume[cols].plot(ax = axes[i,1])
    axes[i,1].set(xlabel='Date', ylabel='Volume')
    axes[i,1].set_title(f"{title}")
    axes[i,1].set_ylim(550000, 450000000)
    monthly_returns[cols].plot(ax = axes[i,2])
    axes[i,2].set(xlabel='Date', ylabel='Monthly Return')
    axes[i,2].set_title(f"{title}")
    axes[i,2].set_ylim(-0.25, 0.5)
    
fig.tight_layout(pad=3.0)

print(colored('\t\t\tMonthly Adjusted Closing VS Monthly Volume VS Monthly % Change', attrs=['bold']))


**In the above graph the first column figure shows monthly adjusted close price patterns, the second column graph shows the monthly volumes traded and the last column shows the monthly returns of each stock.**

In [ ]:
# Visualization of Average Monthly Return
plt.figure(figsize=(12, 7))
plt.bar(columns, monthly_returns.mean())

# zip joins x and y coordinate in pair
for x,y in zip(columns,monthly_returns.mean()):
    label = "{:.3f}".format(y)
    plt.annotate(label,(x,y),textcoords = 'offset points',xytext = (0,10),ha = 'center')

plt.xlabel('Company Symbols')
plt.ylabel('Average monthly return')
plt.suptitle('Average Monthly Return', fontweight = 'bold', fontsize = 14)
plt.show()

**What we can infer after looking at the average monthly return chart is that Amazon has the highest monthly average return with 3.5 percent, while ExxonMobil has the lowest monthly average return with -0.4 percent.**

**Let's filter the stocks down to top 5 with a higher monthly average return and look at the graph.**

In [ ]:
# Visualization of top five stock with highest monthly return
top_five = monthly_returns.mean().nlargest(5)

company_name = list(top_five.keys())
names = []
for key in company_name:
    names.append(CompanyNameMap[key])

plt.figure(figsize=(8, 5))
plt.bar(names, top_five)

# zip joins x and y coordinate in pair
for x,y in zip(names,top_five):
    label = "{:.3f}".format(y)
    plt.annotate(label,
                (x,y),
                textcoords = 'offset points',
                xytext = (0,10),
                ha = 'center')
plt.suptitle('Top 5 companies with highest average monthly return', fontsize = 14, fontweight = 'bold')
plt.ylabel('Average Monthly returns')
plt.xlabel('Companies name')
plt.show()

**Now that we've filtered the companies to top 5 on the basis of average monthly return and have seen the visualizations for the Adjusted price and the volume traded each day with monthly percentage change, let's go ahead and calculate the moving average for the stock.**

# 3. What was the moving average of the various stocks?

**We will be adding three major moving average intervals to our graphs in order to allow us to trade better. They are the moving averages of 10, 20 and 50 days.**

In [ ]:
# Using loop for grabing yahoo data and setting it in form of dataframe
topFiveDataDict={}
for stock in company_name:
    topFiveDataDict[stock] = web.DataReader(stock,"yahoo",'2015-01-01',datetime.now())
    

In [ ]:
#Here we are making pandas dataframe for top 5 company stocks
company_list = topFiveDataDict.values()
for company, comp_name in zip(company_list,company_name):
    company["company_name"] = comp_name
    
company_data = pd.concat(company_list,axis=0)
company_data.tail(5)


In [ ]:
# Calculating the moving averages
ma_day = [10, 20, 50]

for ma in ma_day:
    for company in company_list:
        column_name = f"MA for {ma} days"
        company[column_name] = company['Adj Close'].rolling(ma).mean()

In [ ]:
# here we are visualising three important moving averages of all the company
    
Total = 5
Columns = 2

# array of company datas
datas = list(topFiveDataDict.values())

# names of the company in array
names = list(topFiveDataDict.keys())

# Compute Rows required
Rows = Total // Columns 
Rows += Total % Columns

# Create a Position index
Position = range(1,Total + 1)
fig = plt.figure(1)
fig.set_figheight(15)
fig.set_figwidth(15)

for k in range(Total):
  # add every single subplot to the figure with a for loop
    ax = fig.add_subplot(Rows,Columns,Position[k])
    ax.plot(datas[k]['Adj Close'])
    ax.plot(datas[k]['MA for 10 days'], color = 'yellow')
    ax.plot(datas[k]['MA for 20 days'], color = 'green')
    ax.plot(datas[k]['MA for 50 days'], color = 'orange')
    ax.set_title(CompanyNameMap[names[k]])   
    ax.legend(['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days'])
    ax.set(xlabel='Date',ylabel='Moving Average')

fig.tight_layout(pad=1.5)
fig.suptitle('Moving Average of top 5 companies', fontsize = 16, fontweight = 'bold', y = 1.02)
plt.show()


From the above graph we can conclude that the average trend is identical to the patterns of its Adjusted Close Price regardless of any number of days. Microsoft seems to be the one that most of the time indicates the rising pattern, while Alibaba has the most fluctuating trends. 

# 4. What is the correlation between different stocks ?

In [ ]:
# Getting Top five company Monthly Adjusted Close price
adjusted_close_five= monthly_adjusted_close[company_name]
adjusted_close_five.head()

In [ ]:
# here we are Making DataFrame which is Monthly % change
returns_five= adjusted_close_five.pct_change()
returns_five.tail()

Here we'll use joinplot to compare the monthly return of Amazon to itself and also between Amazon to Microsoft. Correlation between Amazon to itself should shows a perfectly linear relation.

In [ ]:
# Comparing Amazon to itself should show a perfectly linear relationship
fig = sns.jointplot('AMZN', 'AMZN', returns_five, kind='scatter', color = "blue")
fig.set_axis_labels('Amazon','Amazon')
;
print(colored('\tCorrelation between Amazon with Amazon', attrs=['bold']))

In [ ]:
# Comparing monthly returns of Amazon and Microsoft
fig = sns.jointplot('AMZN', 'MSFT', returns_five, kind='scatter', color = "blue")
fig.set_axis_labels('Amazon','Microsoft')
;
print(colored('\tCorrelation between Amazon with Microsoft', attrs=['bold']))

We can see that if two stocks are perfectly (and positivley) correlated with each other a linear relationship between its monthly return values should occur. So now let see the relation between all the stock with each other.

In [ ]:
# Renaming the columns 
fullname = []
for name in company_name:
    fullname.append(CompanyNameMap[name])

returns_five.columns = fullname


In [ ]:
# Here we are calling pairplot on our DataFrame for an automatic visual analysis of all the comparisons
sns.pairplot(returns_five)
;
print(colored('\t\t\t\t\tCorrelation between 5 companies', attrs=['bold']))

It is interesting to see that all stocks are positively correlated and that Microsoft and Amazon had the highest monthly stock return correlation, while Alibaba and Apple were the least correlated. Let's see its meaning in numerical terms to understand this better.

In [ ]:
# Here we are using seaborn for a quick correlation plot for the monthly returns
plt.figure(figsize = (10,5))
sns.heatmap(returns_five.corr(), annot=True, cmap="YlGnBu")
plt.title('Correlation between 5 companies', fontsize = 14, fontweight = 'bold', y = 1.05)
;

# 5. How much value do we put at risk by investing in a particular stock?

It cannot be fruitful to invest in a stock that focuses only on higher returns, ignoring its risk factor. The best investment decision is known to be inventory with high return with low risk. So we'll look at the volatility of the stock from now on as well.

In [ ]:
#Visualiation of Monthly Return percent

#Getting columns heading
columns = list(returns_five) 

plt.figure(figsize=(12, 15))

for i, col in enumerate(columns, 1):
    plt.subplot(3, 2, i)
    plt.plot(returns_five[col])
    plt.ylabel('Monthly Return')
    plt.title(f'{CompanyNameMap[names[i - 1]]}')
    plt.ylim(-0.25,0.45)
    
plt.suptitle('Monthly Return Data\n\n', fontweight = 'bold', fontsize = 14, y = 0.93)
plt.tight_layout(pad=3.0)

After looking at the monthly returns chart of each stock, we can conclude that the returns are quite volatile and the stock can move +0.5  to - 0.3% on any given month. To get a sense of how extreme the returns can be we can plot a histogram.

In [ ]:
#Visualiation of Monthly Return percent
plt.figure(figsize=(12, 15))

for i, col in enumerate(columns, 1):
    plt.subplot(3, 2, i)
    sns.distplot(returns_five[col].dropna(), bins=100, color='blue')
    plt.ylabel('Monthly Return')
    plt.xlabel('Monthly Return %')
    plt.title(f'{CompanyNameMap[names[i - 1]]}')
    plt.ylim(0,25)
    plt.xlim(-0.4,0.65)
    plt.text(-0.3, 15, 'Extreme Low\n Return', fontsize = 12)
    plt.text(0.3, 15, 'Extreme high\n Return', fontsize = 12)
    
plt.suptitle('Monthly Return Data\n\n', fontweight = 'bold', fontsize = 14, y = 0.93)
plt.tight_layout(pad=3.0)

From above we can see that Microsoft's monthly return is least volatile and while Alibaba's is highly volatile.

Plotting monthly returns are useful for understanding the monthly volatility of the investment. To calculate the growth of our investment or in other word, calculating the total returns from our investment, we need to calculate the cumulative returns from that investment. To calculate the cumulative returns we will use the cumprod() function.

In [ ]:
#Calculating the cumulative return
cum_returns = (returns_five + 1).cumprod()
values = cum_returns.tail(1).values[0]
values = map(lambda x : "$ {:.2f}".format(x), values)

#Visualizing the cumulative return of stock
ax = cum_returns.plot(figsize = (10,8))
ax.set_title("Monthly cumulative returns", fontsize = 14, fontweight = 'bold', y = 1.05)
ax.set_xlabel("Date")
ax.set_ylabel("Growth of $1 investment")
for line, name in zip(ax.lines, values):
    y = line.get_ydata()[-1]
    ax.annotate(name, xy=(1,y), xytext=(6,0), color=line.get_color(), 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=14, va="center")

plt.show()

This chart displays the cumulative returns of each stock since 2015. It is showing how much one could have made today with an investment of $1 in 2015. Not surprisingly, from 2015, Amazon has had the highest returns. In the distant second and third, Microsoft and Apple come in. While its lowest earner is Alibaba.

In [ ]:
# Here we are defining a new DataFrame as a cleaned version of the original DataFrame
rets = returns_five.dropna()

area = np.pi*20

#Visualizing the cumulative return of stock
plt.figure(figsize=(7, 5))
plt.scatter(rets.mean(), rets.std(), s=area)
plt.xlabel('Expected return')
plt.ylabel('Risk')
plt.title('Risk and Expected Return',fontsize = 14, fontweight = 'bold', y = 1.25)

for label, x, y in zip(rets.columns, rets.mean(), rets.std()):
    plt.annotate(label, xy=(x, y), xytext=(50, 50), textcoords='offset points', ha='right', va='bottom', 
                 arrowprops=dict(arrowstyle='-', color='blue', connectionstyle='arc3,rad=-0.3'))

Above diagram shows the risk-return relationship of the best five high monthly return stock. It shows that Alibaba, Facebook and Apple Inc has high risk and low return compared to Microsoft and Amazon,  which clearly eliminate this stock on the proposed  investment decision. Amazon showing the highest return stock among them with 3.4% monthly return but considering the risk of 8.5% shows that the Amazon is good stock for investment for aggressive risk taker. As the conservative risk taker, Microsoft is the best stock with low level of risk of 6% to obtain the return of 2.8% is best stock to invest among these stock.

# 6. Portfoilo optimization using Efficient Frontier?

Here we are using the same top five stock with a higher return. We allocated equal weights of 20 percent to each of for the formation of the portfolio and measure the return and risk of investing in that portfolio. After that, using the minimum Sharpe ratio and minimum variance (risk or standard deviation), we determined the effective frontier portfolio.

In [ ]:
# Getting Adjuested Close Price of five selected sock
adjusted_close = data['Adj Close'][company_name]

# Changing the columns name
adjusted_close.columns = fullname
adjusted_close.head()

In [ ]:
# Calculating the daily return
daily_return = adjusted_close.pct_change()
daily_return.head()

In [ ]:
#Analysing of Annual portfiolo return and risk assuming 20% weight on each stock
#assuming trading days = 252 days in a year

weights = np.array([0.2,0.2,0.2,0.2,0.2])

# Calculating annual return of our portfolio
portfolio_return = np.sum(daily_return.mean()* weights)*252

# Calculation of covariance matrix
cov_matrix_annual = daily_return.cov()*252


In [ ]:
# Visualization of covariance matrix
plt.figure(figsize = (10,5))
sns.heatmap(cov_matrix_annual, annot=True, cmap="YlGnBu")
plt.title('Covariance of 5 companies', fontsize = 14, fontweight = 'bold', y = 1.05)
;

**From the above covariance matrix we can conclude that among the top five stocks, Alibabas companys stock has the highest volatility and combination of Alibaba and Appple has least volatility.**

In [ ]:
# Calculating the volatility of our portfolio
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix_annual,weights))
portfolio_std = np.sqrt(portfolio_variance)

# Assuming the value of risk free rate that already exist in market
risk_free_rate = 0.0178

# Calculation of sharpe ratio
# Sharpe ratio shows how much additional return an investor earns by taking additional risk
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std

So, if we invest 20% of our total on each asset, then our portfolio will give:

In [ ]:
print("Expected annual return: " + str(round((portfolio_return * 100),2))+'%')
print("Expected Volatility: " + str(round((portfolio_std * 100),2))+'%')
print("Sharpe Ratio: " + str(round((sharpe_ratio),2)))

We now know the projected return and volatility of our current portfolio. But what if we are not satisfied with our current portfolio's level of volatility and would like to reduce it? In pursuit of a higher expected return, what if we are willing to take on more risk? To achieve these objectives, how do we rearrange the weight of each stock in our portfolio? In this project, our main objective is to obtain an optimal portfolio. Two portfolios that we may like to highlight are
    
    1. The portfolio with the highest Sharpe Ratio (i.e. the highest risk adjusted returns) and 
    2. The "minimum variance portfolio," which is the lowest volatility portfolio

In [ ]:
#calculate mean daily return and covariance of daily returns
mean_daily_returns = daily_return.mean()
cov_matrix = daily_return.cov()

#set number of runs of random portfolio weights
num_portfolios = 15000

In [ ]:
# function that will calculate the annual return and volatility
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns
  
# function that will generate portfolios with random weight
def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(5)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

#Sharpe ratio describes how much excess return you are receiving for the extra volatility that you endure for holding a riskier asset. 
# function to calculate minimum sharpe ratio
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

#function to calculate maximum sharpe ratio
def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# function to calculate annual volatility
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

# function to calculate minimum variance
def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result

# function that calculate the most efficient portfolio for a gieb target return
def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)# Sequential Least Squares Programming (SLSQP).
    return result

# function that will take a range of target return ana compute efficient portfolio of each return level
def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

Let’s try to plot the portfolio choices with maximum Sharpe ratio and minimum volatility also with all the randomly generated portfolios. But this time, we are not picking the optimal ones from the randomly generated portfolios, but we are actually calculating by using Scipy’s ‘minimize’ function. And the below function will also plot the efficient frontier line.

In [ ]:
def display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, _ = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=adjusted_close.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=adjusted_close.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print("-"*80)
    print(colored('Maximum Sharpe Ratio Portfolio Allocation\n', attrs=['bold']))
    print("Annualised Return:" + str(round(rp*100,2)) + "%")
    print("Annualised Volatility:" + str(round(sdp*100,2)) + "%")
    print("\n")
    print(max_sharpe_allocation)
    print("-"*80)
    print(colored('Minimum Volatility Portfolio Allocation\n', attrs=['bold']))
    print("Annualised Return:" + str(round(rp_min * 100,2)) + "%")
    print("Annualised Volatility:" + str(round(sdp_min * 100,2)) + "%")
    print("\n")
    print(min_vol_allocation)
    
    plt.figure(figsize=(10, 7))
   

    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='RdYlBu', marker='o', s=25, alpha=0.3)
    plt.colorbar(label = 'Sharpe ratio')
    plt.scatter(sdp,rp,marker=(5,1,0),color='r',s=100, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker=(5,1,0),color='g',s=100, label='Minimum volatility')

    target = np.linspace(rp_min, 0.42, 30)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='--', color='black', label='efficient frontier')
    plt.title('Calculated Portfolio Optimization based on Efficient Frontier', fontsize = 14, fontweight = 'bold', y = 1.05)
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)

In [ ]:
# Displaying the information and visualaizating of portfolios
display_calculated_ef_with_random(mean_daily_returns, cov_matrix, num_portfolios, risk_free_rate)

For the low risk portfolio, we can see that Microsoft and Apple are given a large budget. If you take another look at the monthly return plot from earlier, you can see that the least volatile stock of five is Microsoft and Apple, so it makes intuitive sense to assign a large percentage to Microsoft and then Apple for the minimum risk portfolio.

If we are able to take a higher risk for a higher return, the maximum Sharpe ratio is one that gives us the best risk-adjusted return. We allocate a large portion to Amazon in this situation, which are fairly volatile stocks from the previous monthly returns map. Microsoft and Apple, which had approximately 50 percent in the case of the low risk portfolio, have allocated the remaining budget to it. Leaving Facebook and Alibaba with 0% alloaction.

We can see that it forms the shape of an arch line on the top of clustered blue dots from the plot of the randomly simulated portfolios. This line is called the efficient frontier. Why is that efficient? Since points along the line will give you the lowest risk of return for a given target return. All the other dots on the line would give you higher risk with the same returns.

The way we found the two kinds of optimal portfolio above was by simulating many possible random choices and pick the best ones (either minimum risk or maximum risk-adjusted return).

Using the minimun volatility portfolio, if we have $10000, lets see much stock we can buy in todays price for the top 5 companies

In [ ]:
# Calculating the expected returns and the annualised sample covariance matrix of asset returns
mu=expected_returns.mean_historical_return(adjusted_close)
S= risk_models.sample_cov(adjusted_close)

In [ ]:
# For minimum volatility
ef=EfficientFrontier(mu,S)
weights=ef.min_volatility()
cleaned_weights=ef.clean_weights()
min_vol_port = ef.portfolio_performance(verbose=True)

In [ ]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(adjusted_close)
weights = cleaned_weights

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

# 7. How can we attempt to predict future stock behavior using LSTM?

In [ ]:
# Getting the data of only Microsoft
df = topFiveDataDict['MSFT']
df.tail()

In [ ]:
#Data cleaning
df.isna().any()
df = df.iloc[:,0:6]
df.isna().any()

In [ ]:
# here we are Visualising the closing price history
plt.figure(figsize=(10,7))
plt.title('Close Price History',fontsize = 14, fontweight = 'bold', y = 1.02)
plt.plot(df['Close'])
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.show()

**From above graph we can see that microsofts closing price is in increasing trend**

In [ ]:
# get close stock data
df1 = df[['Close']]
#reset index of data frame
df1.reset_index(inplace=True)
#converting date to pandas datetime format
df1['Date'] = pd.to_datetime(df1['Date'])

# Creating pivot table for monts
month_table = pd.pivot_table(df1,index=df1['Date'].dt.strftime('%b'),
               values=["Close"],
               aggfunc='mean',fill_value=0)

# Sorting by month
month = ['Jan', 'Feb', 'Mar', 'Apr','May','Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']
month_table.index = pd.CategoricalIndex(month_table.index, categories=month, ordered=True)
month_table = month_table.sort_index()


# Creating pivot table for weeks
weekdays_table = pd.pivot_table(df1,index=df1['Date'].dt.strftime('%a'),
               values=["Close"],
               aggfunc='mean',fill_value=0)

weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
weekdays_table.index = pd.CategoricalIndex(weekdays_table.index, categories=weekdays, ordered=True)
weekdays_table = weekdays_table.sort_index()


In [ ]:
values = list(month_table['Close'].values)

In [ ]:
fig, ax = plt.subplots(1, 2,figsize=(10,5))

month_table.plot(kind = 'bar', ax = ax[0], width=0.75)
ax[0].set_title('Average Monthly Price', fontweight = 'bold', fontsize = 15, y = 1.02)
ax[0].set_xlabel('Months')
ax[0].set_ylabel('Price')
ax[0].get_legend().remove()

for rect in ax[0].patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Use Y value as label and format number with one decimal place
        label = "{:.1f}".format(y_value)

        # Create annotation
        ax[0].annotate(label,                    
            (x_value, y_value),         
            xytext=(0, 3),         
            textcoords="offset points", 
            ha='center') 



weekdays_table.plot(kind = 'bar', ax = ax[1])
ax[1].set_title('Average Weekdays Price', fontweight = 'bold', fontsize = 15, y = 1.02)
ax[1].set_xlabel('Weekdays')
ax[1].set_ylabel('Price')
ax[1].get_legend().remove()

# Code for displaying values above bar
for rect in ax[1].patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Use Y value as label and format number with one decimal place
        label = "{:.1f}".format(y_value)

        # Create annotation
        ax[1].annotate(label,                    
            (x_value, y_value),         
            xytext=(0, 3),         
            textcoords="offset points", 
            ha='center')              

plt.subplots_adjust(bottom=0.1, right=1.8, top=0.9)
plt.show()

From above weekly graph we can see that stock price is are tentatively similar through out the week days but is slighly higher in Monday.

If we look at monthly graph we can see that november is the month with the highest stock price.





Create a new data frame with only the closing price and convert it to an array. Then create a variable to store the length of the training data set. I want the training data set to contain about 80% of the data.

In [ ]:
#Creating a new dataframe with only the 'Close' column
data = df.filter(['Close'])

#Converting the dataframe to a numpy array
dataset = data.values

#Get /Compute the number of rows to train the model on
training_data_len = math.ceil( len(dataset) *.8)


Now scale the data set to be values between 0 and 1 inclusive, I do this because it is generally good practice to scale your data before giving it to the neural network.

In [ ]:
# Feature Scaling, Here we are Scaling the all of the data to be values between 0 and 1 using MinMax Scaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(dataset)

Now we will create a data structure with 60 timestep and 1 output and then split the data into x_train and y_train data sets. Basically what we are trying to do here is take data from day 1 to 60 and make prediction of 61th days

In [ ]:
#Creating the scaled training data set
train_data = scaled_data[0:training_data_len  , : ]

timestep = 60
x_train=[]
y_train = []

#Splitting data into x_train and y_train
for i in range(timestep,len(train_data)):
    x_train.append(train_data[i-timestep:i,0])
    y_train.append(train_data[i,0])
    
#Here we are Converting x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Here we are reshaping the data into the shape accepted by the LSTM
x_train_reshape = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
# Here we are testing data set and again repeat the same process that we did for x_train
test_data = scaled_data[training_data_len - timestep: , : ]

#Creating the x_test and y_test data sets
x_test = []
y_test =  dataset[training_data_len : , : ] # Here we fit the actual data that is lastest 30% data 

for i in range(timestep,len(test_data)):
    x_test.append(test_data[i-timestep:i,0])
    
# here we are converting x_test to a numpy array  
x_test = np.array(x_test)

# here we are reshaping the data into the shape accepted by the LSTM  
x_test_reshape = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
#Now we are Building the LSTM network model
# create model
model = Sequential()

#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 128, return_sequences=True, activation='tanh', input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

#Adding the second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 64, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))

#Adding the third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 32, activation='tanh'))
model.add(Dropout(0.2))

# Addding the output layer
model.add(Dense(1))

In [ ]:
# Compiling the model
model.compile(loss='mse', optimizer='adam')

In [ ]:
# Display the model summary
model.summary()

In [ ]:
# Here we are training the model
val_dataset = tf.data.Dataset.from_tensor_slices((x_test_reshape, y_test)).batch(batch_size=64)
history = model.fit(
    x_train_reshape,
    y_train,
    epochs=100,
    batch_size=64,
    validation_data=val_dataset,
    use_multiprocessing=True
)

In [ ]:
# now we are getting the models predicted price values
predictions = model.predict(x_test_reshape)

predictions = scaler.inverse_transform(predictions)#Undo scaling

In [ ]:
#Create the data for the graph
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

# Table of predicted and actual price
valid.tail()

In [ ]:
#Visualize the data
plt.figure(figsize=(15,7))
plt.title('Model generated price prediction',fontsize = 14, fontweight = 'bold', y = 1.02)
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train['Close'])
plt.plot(valid['Close'], color = 'green')
plt.plot(valid['Predictions'], color = 'orange')
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.title('Predicted Price vs Actual Price',fontsize = 14, fontweight = 'bold', y = 1.02)
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(valid['Close'], color = 'green')
plt.plot(valid['Predictions'], color = 'orange')
plt.legend(['Actual Price', 'Predictions Price'], loc='lower right')
plt.show()
;

print('Mean Absolute Error: {:.3f}'.format(metrics.mean_absolute_error(y_test,predictions)))
print('Mean Square Error: {:.3f}'.format(metrics.mean_squared_error(y_test,predictions)))
print('Root Mean Square Error: {:.3f}'.format(math.sqrt(metrics.mean_squared_error(y_test,predictions))))

What we can infer from the above graph is that our model does the fine job of predicting the stock price as the real price and the expected price are very similar with just 6.46 percent of the mean absolute error 

As we have already developed our model using LSTM for stock price prediction. Now, what we are planning to try is to forecast stock prices for the next 30 days.

In [ ]:
# Taking previous 60 data in order to predict future stock price
x_input = test_data[len(test_data)-timestep:].reshape(1,-1)

temp_input = list(x_input)
temp_input = temp_input[0].tolist()

In [ ]:
# demonstrate prediction for next 30 days

lst_output=[] # list to store the 30 days predicted price
n_steps = 60
i = 0
while(i < 30):
    
    if(len(temp_input) > 60):
        x_input=np.array(temp_input[1:])
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input) # predicting the price
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i + 1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input) # predicting the price
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i = i + 1
    

In [ ]:
future_predictions = scaler.inverse_transform(lst_output)#Undo scaling

In [ ]:
import datetime

lst_date = [] # list to that store next 30 days
day = datetime.datetime.today() # today date

# Getting next 30 days
for i in range(30):
    day = day + datetime.timedelta(days=1)
    nextDay = day.strftime ('%Y-%m-%d') # format the date to ddmmyyyy
    lst_date.append(nextDay)

In [ ]:
# Getting todays date
today = datetime.datetime.today().strftime ('%Y-%m-%d')

# Getting today price of stock
today_price = df['Close'].tail(1).values

# Creating dataframe today stock price
today_df = pd.DataFrame({'Date': today, 'Price':today_price})

In [ ]:
# Adding the predicted stock price to dataframe
pred_df = pd.DataFrame({'Date':lst_date})

pred_df['Price'] = future_predictions

pred_df = pd.concat([today_df,pred_df])

pred_df['Date'] = pd.to_datetime(pred_df['Date'])

pred_df = pred_df.set_index('Date')

# Looking at the head of predicted price
pred_df.head()


In [ ]:
#Visualize the price data
plt.figure(figsize=(16,8))
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(df['Close'].tail(150))
plt.plot(pred_df)
plt.legend(['Past Price', 'Future Price'], loc='upper left')
plt.show()

Thus, our model prediction shows that the stock price will be in increasing trend for the next 30 days. 

# Conclusion

Outline of what we did in this project
- Took 20 companies stock data since 2013 from yahoo finance
- Filtered 20 companies to top 5 on the basis of monthly return on investment
- Made optional portfolio of top 5 companies using [efficient frontier](https://www.investopedia.com/terms/e/efficientfrontier.asp#:~:text=The%20efficient%20frontier%20is%20the,for%20the%20level%20of%20risk.)
- Selected one company among the top 5 on the basis of minimum risk and maximun return
- Used LSTM Neural Network model to predict the price of stock for next 30 days